In [ ]:
from ipywidgets import interact
from matplotlib import cm
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import kde
import numpy as np
import seaborn as sns

In [ ]:
# Functional to be extremized:
F = lambda y, dy: np.sqrt(1+dy**2)

x_min = 0.
x_max = 10.
y_min = -10.
y_max = 10.
dy_min = -10.
dy_max = 10.

y1 = -4.
y2 = 8.

slope = (y2 - y1) / (x_max - x_min)
y_func1 = lambda x_: slope * x_ + y1
dy_func1 = lambda x_: np.ones(np.shape(x_)) * slope
omega = 2. * np.pi / (x_max - x_min) * 2
eta = lambda x_: np.sin((x_ - x_min) * omega)
d_eta = lambda x_: omega * np.cos((x_ - x_min) * omega)
y_func2_e = lambda x_, epsilon_: y_func1(x_) + epsilon_ * eta(x_)
dy_func2_e = lambda x_, epsilon_: dy_func1(x_) + epsilon_ * d_eta(x_)

X = np.linspace(x_min, x_max, 60)
Y, DY = np.meshgrid(
    np.linspace(y_min, y_max, 40),
    np.linspace(dy_min, dy_max, 40),
)
Epsilon = np.linspace(-3., 3., 30)
L = F(Y, DY)

I = lambda epsilon_: np.sum(F(y_func2_e(X, epsilon_), dy_func2_e(X, epsilon_)))

I_values = []
for epsilon_ in Epsilon:
    I_values.append(I(epsilon_))

@interact(
    view_angle=(0., 360., 10.),
    x=(x_min, x_max),
    #y1=(y_min, y_max),
    #y2=(y_min, y_max),
    epsilon=(-3., 3.),
)
def f(
    view_angle=120.,
    x=4.,
    #y1=-4.,
    #y2=8.,
    epsilon=1.5,
):
    fig, ((ax2, ax), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18,10))
    ax.axis('off')
    ax = fig.add_subplot(2, 2, 2, projection='3d')

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('dy')
    ax.view_init(30, -view_angle)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_zlim(dy_min, dy_max)
    
    ax.plot_surface(np.ones(Y.shape) * x, Y, DY, rstride=1, cstride=1, facecolors=plt.cm.coolwarm(L/L.max()), shade=False, alpha=0.2)
    ax.plot([x_min, x_min], [y1, y1], [dy_min, dy_max], color='k', lw=2)
    ax.plot([x_max, x_max], [y2, y2], [dy_min, dy_max], color='k', lw=2)
    
    y_func2 = lambda x_: y_func2_e(x_, epsilon)
    dy_func2 = lambda x_: dy_func2_e(x_, epsilon)
    
    Y1 = y_func1(X)
    DY1 = dy_func1(X)
    Y2 = y_func2(X)
    DY2 = dy_func2(X)
    ax.plot(X, Y1, DY1, color='g')
    ax.plot(X, Y2, DY2, color='r')
    ax.plot([x], [y_func1(x)], [dy_func1(x)], 'o', color='g')
    ax.plot([x], [y_func2(x)], [dy_func2(x)], 'o', color='r')
    
    ax2.set_title('y(x)')
    ax2.plot(X, Y1, color='g')
    ax2.plot(X, Y2, color='r')
    
    #ax3.set_title('F(x, y, y\')')
    #ax3.plot(X, F(y_func1(X), dy_func1(X)), color='g')
    #ax3.plot(X, F(y_func2(X), dy_func2(X)), color='r')
    #ax3.set_ylim(0., 5.)
    
    ax3.set_title('∫F(x, y, y\')')
    ax3.plot(X, np.cumsum(F(y_func1(X), dy_func1(X))), color='g')
    ax3.plot(X, np.cumsum(F(y_func2(X), dy_func2(X))), color='r')
    ax3.set_ylim(0., 200.)
    
    ax4.set_title('I(ε)')
    ax4.plot(Epsilon, I_values)
    ax4.plot(epsilon, I(epsilon), 'o', color='k')